In [7]:
import glob
import nbformat
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import shutil
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
import hashlib
import sys
import platform
from bs4 import BeautifulSoup
import re
import copy
import os

In [3]:
fpaths = glob.glob('../../accy575-sp2023/05-yellow-cab-submissions/late-submissions/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['../../accy575-sp2023/05-yellow-cab-submissions/late-submissions\\heyangz2-yellow-cab-case.ipynb',
 '../../accy575-sp2023/05-yellow-cab-submissions/late-submissions\\meighan2-05-yellow-cab-casePM.ipynb']

In [3]:
fpaths = glob.glob('test-notebooks/quiz-02-20230919-v1-test.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['test-notebooks/quiz-02-20230919-v1-test.ipynb']

In [4]:
fpaths = glob.glob('test-noteboks/05-yellow*-SOLUTION.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['test-notebooks\\05-yellow-cab-case-SOLUTION.ipynb']

In [8]:
fpaths = glob.glob('../submissions/case-study-04/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['../submissions/case-study-04\\acevedomaya_LATE_69367_10488185_Case_Study_4.ipynb',
 '../submissions/case-study-04\\acharyanamrata_78686_10471452_case_study_04_rideshare_trips.ipynb',
 '../submissions/case-study-04\\adikareem_334089_10473554_case_study_04_rideshare_trips.ipynb']

In [18]:
fpaths_1 = glob.glob('../../accy575-sp2023/05-yellow-cab-submissions/AB-lambdagrader/*heyang*.ipynb')
fpaths_2 = glob.glob('../../accy575-sp2023/05-yellow-cab-submissions/AB-lambdagrader/meighan*.ipynb')
fpaths = [*fpaths_1, *fpaths_2]
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['../../accy575-sp2023/05-yellow-cab-submissions/AB-lambdagrader\\zhangheyang_LATE_4775_8297034_heyangz2-yellow-cab-case.ipynb',
 '../../accy575-sp2023/05-yellow-cab-submissions/AB-lambdagrader\\meighanphilip_LATE_6126_8427062_05-yellow-cab-casePM.ipynb']

In [9]:
import lambdagrader

graded_file_paths = fpaths
graded_results = []
file_index = 0

for notebook_path in graded_file_paths:
    try:
        print('=============================')
        nb = nbformat.read(notebook_path, as_version=4)
        
        test_cases_hash = lambdagrader.get_test_cases_hash(nb)
        
        lambdagrader.preprocess_test_case_cells(nb)
        lambdagrader.add_grader_scripts(nb)

        p = Path(notebook_path)
        filestem = p.name
        
        print(f'Grading {notebook_path}')
        
        ep = ExecutePreprocessor(
            timeout=1800,
            kernel_name='python3',
            allow_errors=True
        )
        ep.preprocess(nb)
        
        # save graded notebook
        converted_notebook_path = notebook_path.replace('.ipynb', '-graded.ipynb')
        with open(converted_notebook_path, mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        
        # running the notebook will store the graded result to a JSON file
        # rename graded result JSON file
        graded_result_json_path = notebook_path.replace('.ipynb', '-result.json')
        shutil.move('lambdagrader-result.json', graded_result_json_path)
        
        # read graded result to generate a summary
        with open(graded_result_json_path, mode='r') as f:
            graded_result = json.load(f)
        
        # add filename
        # we add it here instead of trying to add it within the Jupyter notebook
        # because it is tricky to grab the current file name inside a Jupyter kernel
        graded_result['filename'] = Path(notebook_path).name
        
        # MD5 hash of the submitted Jupyter notebook file
        # this can be used to detect duplicate submission to prevent unnecessary re-grading
        with open(notebook_path, 'rb') as f:
            graded_result['submission_notebook_hash'] = hashlib.md5(f.read()).hexdigest()
        
        # MD5 hash of test cases code
        # this helps us to identify any potential cases
        # where a learner has modified or deleted the test cases code cell
        graded_result['test_cases_hash'] = test_cases_hash
        
        # store Python version and platform used to run the notebook
        graded_result['grader_python_version'] = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
        graded_result['grader_platform'] = platform.platform()

        # save updated JSON to file
        with open(graded_result_json_path, 'w') as f:
            json.dump(graded_result, f, indent=2)
            
        # clean up notebook
        lambdagrader.remove_grader_scripts(nb)
        lambdagrader.add_graded_result(nb, graded_result)
           
        # extract user code to a Python file
        extracted_user_code = lambdagrader.extract_user_code_from_notebook(nb)
        extracted_code_path = notebook_path.replace('.ipynb', '_user_code.py')
        
        with open(extracted_code_path, "w", encoding="utf-8") as f:
            f.write(extracted_user_code)
        
        # store graded result to HTML
        filestem = Path(notebook_path).name
        graded_html_path = notebook_path.replace('.ipynb', '-graded.html')
        lambdagrader.save_graded_notebook_to_html(
            nb,
            html_title=filestem,
            output_path=graded_html_path,
            graded_result=graded_result
        )
        
        # LOCAL ENVIRONMENT ONLY
        # the Lambda handler only processes one file instead of
        # running a batch
        # the code below generates a CSV to show results for multiple files
        # get text summary of user's graded result
        text_summary = lambdagrader.generate_text_summary(graded_result)
        
        result_summary = graded_result.copy()
        result_summary['text_summary'] = text_summary
        del result_summary['results']
        graded_results.append(result_summary)
        
        print(f"[{round((file_index + 1) / len(graded_file_paths) * 100, 1)}%] Graded {file_index + 1} out of {len(graded_file_paths)} files")
        file_index += 1
    except Exception as e:
        print(f'Error while grading {notebook_path}')
        print('-----------------------------')
        print(e)
        
df_summary = pd.DataFrame(graded_results)
df_summary.sort_values(by=['learner_autograded_score', 'test_cases_hash'], inplace=True)

df_summary.to_csv(
    f"graded_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
    index=None
)

Grading ../submissions/case-study-04\acevedomaya_LATE_69367_10488185_Case_Study_4.ipynb
[0.9%] Graded 1 out of 110 files
Grading ../submissions/case-study-04\acharyanamrata_78686_10471452_case_study_04_rideshare_trips.ipynb
[1.8%] Graded 2 out of 110 files
Grading ../submissions/case-study-04\adikareem_334089_10473554_case_study_04_rideshare_trips.ipynb
[2.7%] Graded 3 out of 110 files
Grading ../submissions/case-study-04\agboifohsomto_89269_10475286_Case_Study_4_Rideshare_Trips.ipynb
[3.6%] Graded 4 out of 110 files
Grading ../submissions/case-study-04\akenouagnes_333199_10474715_Copy_of_case_study_04_rideshare_trips.ipynb
[4.5%] Graded 5 out of 110 files
Grading ../submissions/case-study-04\ambergben_LATE_115049_10489800_case_study_04_rideshare_trips.ipynb
[5.5%] Graded 6 out of 110 files
Grading ../submissions/case-study-04\ayubhiba_330946_10450146_case_study_04_rideshare_trips.ipynb
[6.4%] Graded 7 out of 110 files
Grading ../submissions/case-study-04\bathinitarun_123641_10470884_c

In [12]:
from copydetect import CopyDetector
detector = CopyDetector(test_dirs=["../submissions/exercise-05"], extensions=["py"])
# detector.run()
# detector.generate_html_report()

  0.00: Generating file fingerprints


   100%|██████████| 224/224 [00:04<00:00, 49.09it/s]


  4.57: Beginning code comparison


   100%|██████████| 112/112 [00:00<00:00, 120.30it/s]


  5.50: Code comparison completed
Output saved to ./report.html


In [9]:
# from lambdagrader import *

notebook_path = '../notebooks\\case-study-04-rideshare-trips-SOLUTION.ipynb'
nb = nbformat.read(notebook_path, as_version=4)

tcs = lambdagrader.extract_test_cases_metadata_from_notebook(nb)

s = 0

for o in tcs:
    s += o['points']
    
print(s)

80.0
